# Deep Learning approach for single-cell RNA-seq data analysis

In [19]:
import os
import time
import warnings
import scanpy as sc
import anndata
import scvi
import torch
import pandas as pd
import numpy as np
import ray
from ray import tune
from scvi import autotune

start_time = time.time()

scvi.settings.seed = 0
scvi.settings._dl_pin_memory_gpu_training = 20000

sc.set_figure_params(figsize=(5, 5))
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor_high_precision = torch.tensor([1.0, 2.0, 3.0], dtype=torch.float64, device=device)
torch.set_float32_matmul_precision('high')

save_dir = os.getcwd()

Global seed set to 0


In [2]:
print("Used versions:")
print("scvi-tools version:", scvi.__version__)
print("scanpy version:", sc.__version__)
print("torch version:", torch.__version__)
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("Save dir:", save_dir)

Used versions:
scvi-tools version: 1.0.3
scanpy version: 1.9.5
torch version: 2.0.1
pandas version: 1.5.3
numpy version: 1.23.5
Save dir: /Users/sayedjm/Documents/Bio-informatica/Studiejaar 3/Bpexa/workflow


In [3]:
data_0h_A = sc.read_10x_h5("data/0h_A_raw_feature_bc_matrix.h5")
data_0h_B = sc.read_10x_h5("data/0h_B_raw_feature_bc_matrix.h5")
data_48h = sc.read_10x_h5("data/48h_raw_feature_bc_matrix.h5")
data_120h = sc.read_10x_h5("data/120h_raw_feature_bc_matrix.h5")

In [4]:
data_0h_A.obs['run'] = "0h_A"
data_0h_B.obs['run'] = "0h_B"
data_48h.obs['run'] = "48h"
data_120h.obs['run'] = "120h"

In [5]:
sc.pp.filter_genes(data_0h_A, min_cells=3)
sc.pp.filter_genes(data_0h_B, min_cells=3)
sc.pp.filter_genes(data_48h, min_cells=3)
sc.pp.filter_genes(data_120h, min_cells=3)

In [6]:
sc.pp.filter_cells(data_0h_A, min_counts=200)
sc.pp.filter_cells(data_0h_B, min_counts=200)
sc.pp.filter_cells(data_48h, min_counts=200)
sc.pp.filter_cells(data_120h, min_counts=200)

In [7]:
data_0h_A.var_names_make_unique()
data_0h_B.var_names_make_unique()
data_48h.var_names_make_unique()
data_120h.var_names_make_unique()

In [8]:
data_list = [data_0h_A, data_0h_B, data_48h, data_120h]
keys = ['batch_2_A', 'batch_2_B', 'batch_1_sample_48H', 'batch_1_sample_120H']
for data, key in zip(data_list, keys):
    data.obs_names = key + '_' + data.obs_names

adata = anndata.concat(data_list, join="outer")

In [9]:
sample_names = pd.read_table("data/sample_name_full_dataset.tsv")
sample_names.rename(columns={'Unnamed: 0': 'Sample', 'sample_name': 'Batch'}, inplace=True)
sample_names.set_index('Sample', inplace=True)
adata.obs['batch'] = adata.obs_names.map(sample_names['Batch'])

In [10]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top= None, log1p=False, inplace=True)

In [11]:
data = pd.DataFrame(data=adata.obs)
data.head()

,run,n_counts,batch,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt
batch_2_A_AAACCCAAGGTAATCA-1,0h_A,48806.0,iPSC_99_4,6892,48806.0,2494.0,5.110028
batch_2_A_AAACCCAAGGTACCTT-1,0h_A,42383.0,iPSC_20_6_4,6574,42383.0,2371.0,5.594224
batch_2_A_AAACCCAGTCGTTCAA-1,0h_A,15016.0,iPSC_54_3,3975,15016.0,881.0,5.867075
batch_2_A_AAACCCATCACAGTGT-1,0h_A,37052.0,iPSC_72_1,6262,37052.0,1978.0,5.338443
batch_2_A_AAACCCATCCCTCTCC-1,0h_A,2387.0,iPSC_72_1,374,2387.0,1977.0,82.823631


In [12]:
subset_condition = (
    (adata.obs['n_genes_by_counts'] > 2000) &
    (adata.obs['n_genes_by_counts'] < 7000) &
    (adata.obs['n_counts'] < 100000) &
    (adata.obs['pct_counts_mt'] < 10) &
    (adata.obs['pct_counts_mt'] > 0.1)
)
adata = adata[subset_condition, :]

## Normalization

In [13]:
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e5)
sc.pp.log1p(adata)
adata.raw = adata

In [14]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    layer="counts",
    flavor='seurat_v3',
    subset=True
)

In [15]:
training_size = 0.9
while np.ceil((len(adata) * training_size) % 128) == 1: training_size -= 0.01
print(f"Current training_size: {training_size}")

Current training_size: 0.89


## Training model

In [16]:
scvi.model.SCVI.setup_anndata(adata, layer="counts", batch_key="batch")

In [17]:
model = scvi.model.SCVI(adata)
#model.train(batch_size = 128, train_size=training_size, max_epochs=100)
model_dir = os.path.join(save_dir, "scvi_model")
#model.save(model_dir)

In [23]:
model_cls = scvi.model.SCVI
model_cls.setup_anndata(adata)
scvi_tuner = autotune.ModelTuner(model_cls)

In [24]:
scvi_tuner.info()

ModelTuner registry for SCVI

                  Tunable hyperparameters                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃      Hyperparameter      ┃ Default value ┃    Source    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│         n_hidden         │      128      │     VAE      │
│         n_latent         │      10       │     VAE      │
│         n_layers         │       1       │     VAE      │
│       dropout_rate       │      0.1      │     VAE      │
│        dispersion        │     gene      │     VAE      │
│     gene_likelihood      │     zinb      │     VAE      │
│   latent_distribution    │    normal     │     VAE      │
│    encode_covariates     │     False     │     VAE      │
│ deeply_inject_covariates │     True      │     VAE      │
│      use_batch_norm      │     both      │     VAE      │
│      use_layer_norm      │     none      │     VAE      │
│        optimizer         │     Adam      │ TrainingPlan │
│            lr            │     0.001     │ TrainingPlan │
│       weight_decay       │     1e-06     │ TrainingPlan │
│           eps            │     0.01      │ TrainingPlan │
│    n_steps_kl_warmup     │     None      │ TrainingPlan │
│    n_epochs_kl_warmup    │      400      │ TrainingPlan │
│   reduce_lr_on_plateau   │     False     │ TrainingPlan │
│        lr_factor         │      0.6      │ TrainingPlan │
│       lr_patience        │      30       │ TrainingPlan │
│       lr_threshold       │      0.0      │ TrainingPlan │
│          lr_min          │       0       │ TrainingPlan │
│      max_kl_weight       │      1.0      │ TrainingPlan │
│      min_kl_weight       │      0.0      │ TrainingPlan │
└──────────────────────────┴───────────────┴──────────────┘

       Available metrics        
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃     Metric      ┃    Mode    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ validation_loss │    min     │
└─────────────────┴────────────┘

                         Default search space                         
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Hyperparameter ┃ Sample function ┃  Arguments  ┃ Keyword arguments ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│    n_hidden    │     choice      │ [[64, 128]] │        {}         │
└────────────────┴─────────────────┴─────────────┴───────────────────┘

In [30]:
search_space = {
    "n_hidden": tune.choice([64, 128, 256]),
    "n_layers": tune.choice([1, 2, 3, 10]),
}

In [26]:
ray.shutdown()
ray.init(log_to_driver=False)

2023-11-20 17:36:09,836	INFO worker.py:1664 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.1
Ray version:,2.8.0
Dashboard:,http://127.0.0.1:8265


In [ ]:
results = scvi_tuner.fit(
    adata,
    metric="validation_loss",
    search_space=search_space,
    num_samples=5,
    max_epochs=100,
    resources={"cpu": 8},
)

2023-11-20 17:45:38,907	INFO tune.py:586 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
2023-11-20 17:45:51,583	ERROR tune_controller.py:1383 -- Trial task failed for trial _trainable_76d05b2e
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
 

In [ ]:
print(results.model_kwargs)
print(results.train_kwargs)

In [ ]:
t_model = scvi.model.SCVI.load("scvi_model/", adata=adata)

In [ ]:
t_model

# PCA umap

In [ ]:
sc.tl.pca(adata, n_comps=15)
sc.pp.neighbors(adata, n_pcs=15)
sc.tl.umap(adata, min_dist=0.3)

In [ ]:
sc.pl.umap(
    adata,
    color=["batch"],
    frameon=False,
)

# latent umap

In [ ]:
adata.obsm["X_scvi"] = t_model.get_latent_representation()
adata.layers['scvi_normalized'] = t_model.get_normalized_expression(libray_size = 1e5)

In [ ]:
sc.pp.neighbors(adata, use_rep="X_scvi")
sc.tl.umap(adata)
sc.pl.umap(
    adata,
    title="hiPSC",
    color=["batch", "run"],
    frameon=False,
    ncols=1
)


In [ ]:
sc.tl.leiden(adata, key_added="leiden_scVI", resolution=0.3)
sc.pl.umap(
    adata,
    color=["leiden_scVI"],
    frameon=False,
)

In [ ]:
t_model.history